In [ ]:
# Untitled

In [32]:
from datetime import datetime, timedelta
from pathlib import Path
import shutil

from adcircpy import AdcircMesh, AdcircRun, Tides
from adcircpy.utilities import download_mesh, get_logger

LOGGER = get_logger(__name__)

DATA_DIRECTORY = Path("/Users/simon/adcirc-swan/adcircpy") / "data"
INPUT_DIRECTORY = DATA_DIRECTORY / "input" / "shinnecock"
OUTPUT_DIRECTORY = DATA_DIRECTORY / "output" / "example_1"

MESH_DIRECTORY = INPUT_DIRECTORY / "shinnecock"

download_mesh(
    url="https://www.dropbox.com/s/1wk91r67cacf132/NetCDF_shinnecock_inlet.tar.bz2?dl=1",
    directory=MESH_DIRECTORY,
    known_hash="99d764541983bfee60d4176af48ed803d427dea61243fa22d3f4003ebcec98f4",
)

# open mesh file
mesh = AdcircMesh.open(MESH_DIRECTORY / "fort.14", crs=4326)

# initialize tidal forcing and constituents
tidal_forcing = Tides()
mesh = AdcircMesh.open(MESH_DIRECTORY / "fort.14", crs=4326)
# initialize atmospheric mesh forcings (for NUOPC coupling)
wind_forcing = AtmosphericMeshForcing(
    filename="Wind_HWRF_SANDY_Nov2018_ExtendedSmoothT.nc",
    nws=11,
    interval_seconds=3600,
)
mesh.add_forcing(wind_forcing)
mesh.add_forcing(tidal_forcing)

# set simulation dates
duration = timedelta(days=5)
start_date = datetime(2015, 12, 14)
end_date = start_date + duration

# instantiate driver object
driver = AdcircRun(mesh, start_date, end_date)

# override default options so the resulting `fort.15` matches the original Shinnecock test case options
driver.run(OUTPUT_DIRECTORY, overwrite=True, nproc=1)

ok
 INFO: The ROOTDIR is  '.'.
 INFO: The INPUTDIR is '.'.
 INFO: The GBLINPUTDIR is '.'.
 INFO: The GLOBALDIR is '.'.
 INFO: The LOCALDIR is '.'.
ERROR: logNameListReadStatus: Could not read metControl namelist. The Fortran i/o error code was 5010.

 _______________________________________________________________________________


                   PROGRAM ADCIRC   VERSION 454104d

     AN ADVANCED CIRCULATION MODEL FOR SHELVES, COASTAL SEAS AND ESTUARIES


       -  DEVELOPED BY

          R.A. LUETTICH, JR
            UNIVERSITY OF NORTH CAROLINA AT CHAPEL HILL
            INSTITUTE OF MARINE SCIENCES

          J.J. WESTERINK 
            DEPARTMENT OF CIVIL ENGINEERING AND GEOLOGICAL SCIENCES
            UNIVERSITY OF NOTRE DAME


       -  THE ADCIRC SOURCE CODE IS COPYRIGHTED BY

          R.A. LUETTICH, JR. AND J.J. WESTERINK, 1994-2006

       NO PART OF THIS CODE MAY BE REPRODUCED OR REDISTRIBUTED
          WITHOUT THE WRITTEN PERMISSION OF THE AUTHORS



 __________________

In [2]:
pwd

'/Users/simon/adcirc-swan/adcircpy'

In [4]:
driver

In [1]:
"""Holland 2010 implementation

@article{holland2010revised,
  title={A revised model for radial profiles of hurricane winds},
  author={Holland, Greg J and Belanger, James I and Fritz, Angela},
  journal={Monthly weather review},
  volume={138},
  number={12},
  pages={4393--4401},
  year={2010},
  doi={10.1175/2010MWR3317.1}
}

"""
import warnings
import matplotlib.pyplot as plt
import numpy as np
from pyschism.forcing.winds.atcf import Bdeck
from scipy import optimize


def holland_B(hurdat, coriolis=True):
    # air_density = 1.225
    air_density = 1.15

    def with_coriolis(Vmax, Rmax, Pn, Pc, eye_lat) -> float:
        """

        Args:
            Vmax (float): "max_sustained_wind_speed"
            Rmax (float): "radius_of_maximum_winds"
            Pn (float): "background_pressure"
            Pc (float): "central_pressure"
            eye_lat (float): eye latitude.

        Returns:
            float: _description_
        """
        f = 2.0 * 7.2921e-5 * np.sin(np.radians(np.abs(eye_lat)))
        return (Vmax**2 + Vmax * Rmax * f * air_density * np.exp(1)) / (Pn - Pc)

    def no_coriolis(Vmax, Pn, Pc):
        return (Vmax**2 * air_density * np.exp(1)) / (Pn - Pc)

    for data in hurdat.values():

        Pc = data["central_pressure"]
        Pn = data["background_pressure"]
        # avoid negative Holland B parameter as initial guess
        if Pn <= Pc:
            Pn = Pc + 1.0
        if coriolis:
            return with_coriolis(
                data["max_sustained_wind_speed"],
                data["radius_of_maximum_winds"],
                Pn,
                Pc,
                data["eye"]["lat"],
            )
        else:
            return no_coriolis(data["max_sustained_wind_speed"], Pn, Pc)


def main():
    storm_id = "AL152017"
    # storm_id = 'AL182012'
    hurdat = Bdeck(storm_id).data

    for time, data in hurdat.items():
        if len(data["isotachs"].keys()) != 4:
            continue
        # initial guesses
        Vmax = data["max_sustained_wind_speed"]
        Rmax = data["radius_of_maximum_winds"]
        # print(data)
        # exit()
        x = 1.0
        B = 1.0

        def holland2010(r, B, x):
            return Vmax * (((Rmax / r) ** B) * np.exp(1 - (Rmax / r) ** B)) ** x

        def V(B, x):
            def v(r):
                return holland2010(r, B, x)

            return v

        # B = holland_B(hurdat)
        for quad, isotachs in data["isotachs"].items():
            xdata = []
            ydata = []
            for y, x in isotachs.items():
                xdata.append(x)
                ydata.append(y)
            # xdata.append(Rmax)
            # ydata.append(Vmax)
            # add bounds
            bi = np.finfo(float).eps  # avoid divide by zero
            bf = data["radius_of_last_closed_isobar"]
            bounds = (bi, bf)
            p0 = [B, x]
            # do curve fitting
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                popt, pcov = optimize.curve_fit(
                    holland2010,
                    xdata,
                    ydata,
                    p0=p0,
                    # bounds=bounds,
                    method="dogbox",
                )
                print(popt)
            v = V(*popt)
            radii = np.linspace(bi, bf, num=500)
            res = []
            for i in radii:
                res.append(v(i))
            results = np.array(res)
            plt.plot(radii, results, label=quad)
            # plt.gca().axis('scaled')

        plt.legend()
        plt.show()
        plt.close(plt.gcf())
        # Vmax
        # res = minimize(holland2010, x0=[Vmax, Rmax, B, 0.1] )
        # print(res)


def init():
    if __name__ == "__main__":
        try:
            import colored_traceback

            colored_traceback.add_hook(always=True)
        except ModuleNotFoundError:
            pass
        main()


init()

ModuleNotFoundError: No module named 'pyschism.forcing.winds'

In [13]:
from datetime import datetime, timedelta
from pathlib import Path

from adcircpy import AdcircMesh, AdcircRun, Tides
from adcircpy.forcing.waves.ww3 import WaveWatch3DataForcing
from adcircpy.forcing.winds.atmesh import AtmosphericMeshForcing
from adcircpy.server import SlurmConfig
from adcircpy.utilities import download_mesh

DATA_DIRECTORY = Path("/Users/simon/adcirc-swan/adcircpy") / "data"
INPUT_DIRECTORY = DATA_DIRECTORY / "input"
OUTPUT_DIRECTORY = DATA_DIRECTORY / "output" / "example_4"

MESH_DIRECTORY = INPUT_DIRECTORY / "shinnecock"

download_mesh(
    url="https://www.dropbox.com/s/1wk91r67cacf132/NetCDF_shinnecock_inlet.tar.bz2?dl=1",
    directory=MESH_DIRECTORY,
    known_hash="99d764541983bfee60d4176af48ed803d427dea61243fa22d3f4003ebcec98f4",
)


# open mesh file
mesh = AdcircMesh.open(MESH_DIRECTORY / "fort.14", crs=4326)


# initialize atmospheric mesh forcings (for NUOPC coupling)
wind_forcing = AtmosphericMeshForcing(
    filename="Wind_HWRF_SANDY_Nov2018_ExtendedSmoothT.nc",
    nws=5,
    interval_seconds=3600,
)
mesh.add_forcing(wind_forcing)


# instantiate driver object
driver = AdcircRun(
    mesh=mesh,
    start_date=datetime.now(),
    end_date=timedelta(days=7),
    spinup_time=timedelta(days=5),
    # server_config=slurm,
)

# write driver state to disk
# driver.write(OUTPUT_DIRECTORY, overwrite=True)
driver.run(OUTPUT_DIRECTORY, overwrite=True, nproc=1)

ok
 INFO: The ROOTDIR is  '.'.
 INFO: The INPUTDIR is '.'.
 INFO: The GBLINPUTDIR is '.'.
 INFO: The GLOBALDIR is '.'.
 INFO: The LOCALDIR is '.'.
ERROR: logNameListReadStatus: Could not read metControl namelist. The Fortran i/o error code was 5010.

 _______________________________________________________________________________


                   PROGRAM ADCIRC   VERSION 454104d

     AN ADVANCED CIRCULATION MODEL FOR SHELVES, COASTAL SEAS AND ESTUARIES


       -  DEVELOPED BY

          R.A. LUETTICH, JR
            UNIVERSITY OF NORTH CAROLINA AT CHAPEL HILL
            INSTITUTE OF MARINE SCIENCES

          J.J. WESTERINK 
            DEPARTMENT OF CIVIL ENGINEERING AND GEOLOGICAL SCIENCES
            UNIVERSITY OF NOTRE DAME


       -  THE ADCIRC SOURCE CODE IS COPYRIGHTED BY

          R.A. LUETTICH, JR. AND J.J. WESTERINK, 1994-2006

       NO PART OF THIS CODE MAY BE REPRODUCED OR REDISTRIBUTED
          WITHOUT THE WRITTEN PERMISSION OF THE AUTHORS



 __________________

 TIME STEP =    4000   2.32% COMPLETE     ITERATIONS =    0     TIME =  0.10043124E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =    4100   2.38% COMPLETE     ITERATIONS =    0     TIME =  0.10294202E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =    4200   2.44% COMPLETE     ITERATIONS =    0     TIME =  0.10545280E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =    4300   2.50% COMPLETE     ITERATIONS =    0     TIME =  0.10796358E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =    4400   2.56% COMPLETE     ITERATIONS =    0     TIME =  0.11047436E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =    4500   2.62% COMPLETE     ITERATIONS =    0     TIME =  0.11298515E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT 

 TIME STEP =    9100   5.29% COMPLETE     ITERATIONS =    0     TIME =  0.22848107E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =    9200   5.35% COMPLETE     ITERATIONS =    0     TIME =  0.23099185E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =    9300   5.41% COMPLETE     ITERATIONS =    0     TIME =  0.23350263E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =    9400   5.46% COMPLETE     ITERATIONS =    0     TIME =  0.23601341E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =    9500   5.52% COMPLETE     ITERATIONS =    0     TIME =  0.23852419E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =    9600   5.58% COMPLETE     ITERATIONS =    0     TIME =  0.24103498E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT 

 TIME STEP =   14200   8.25% COMPLETE     ITERATIONS =    0     TIME =  0.35653090E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =   14300   8.31% COMPLETE     ITERATIONS =    0     TIME =  0.35904168E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =   14400   8.37% COMPLETE     ITERATIONS =    0     TIME =  0.36155246E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =   14500   8.43% COMPLETE     ITERATIONS =    0     TIME =  0.36406325E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =   14600   8.49% COMPLETE     ITERATIONS =    0     TIME =  0.36657403E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =   14700   8.54% COMPLETE     ITERATIONS =    0     TIME =  0.36908481E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT 

 TIME STEP =   19300  11.22% COMPLETE     ITERATIONS =    0     TIME =  0.48458073E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =   19400  11.28% COMPLETE     ITERATIONS =    0     TIME =  0.48709151E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =   19500  11.33% COMPLETE     ITERATIONS =    0     TIME =  0.48960230E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =   19600  11.39% COMPLETE     ITERATIONS =    0     TIME =  0.49211308E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =   19700  11.45% COMPLETE     ITERATIONS =    0     TIME =  0.49462386E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =   19800  11.51% COMPLETE     ITERATIONS =    0     TIME =  0.49713464E+05
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT 

...

 TIME STEP =  156600  91.02% COMPLETE     ITERATIONS =    0     TIME =  0.39318830E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =  156700  91.07% COMPLETE     ITERATIONS =    0     TIME =  0.39343938E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =  156800  91.13% COMPLETE     ITERATIONS =    0     TIME =  0.39369046E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =  156900  91.19% COMPLETE     ITERATIONS =    0     TIME =  0.39394154E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =  157000  91.25% COMPLETE     ITERATIONS =    0     TIME =  0.39419262E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =  157100  91.31% COMPLETE     ITERATIONS =    0     TIME =  0.39444370E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT 

 TIME STEP =  161700  93.98% COMPLETE     ITERATIONS =    0     TIME =  0.40599329E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =  161800  94.04% COMPLETE     ITERATIONS =    0     TIME =  0.40624437E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =  161900  94.10% COMPLETE     ITERATIONS =    0     TIME =  0.40649544E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =  162000  94.15% COMPLETE     ITERATIONS =    0     TIME =  0.40674652E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =  162100  94.21% COMPLETE     ITERATIONS =    0     TIME =  0.40699760E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =  162200  94.27% COMPLETE     ITERATIONS =    0     TIME =  0.40724868E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT 

 TIME STEP =  166800  96.94% COMPLETE     ITERATIONS =    0     TIME =  0.41879827E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =  166900  97.00% COMPLETE     ITERATIONS =    0     TIME =  0.41904935E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =  167000  97.06% COMPLETE     ITERATIONS =    0     TIME =  0.41930043E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =  167100  97.12% COMPLETE     ITERATIONS =    0     TIME =  0.41955151E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =  167200  97.18% COMPLETE     ITERATIONS =    0     TIME =  0.41980258E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =  167300  97.23% COMPLETE     ITERATIONS =    0     TIME =  0.42005366E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT 

 TIME STEP =  171900  99.91% COMPLETE     ITERATIONS =    0     TIME =  0.43160325E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
 TIME STEP =  172000  99.97% COMPLETE     ITERATIONS =    0     TIME =  0.43185433E+06
  ELMAX =  0.0000E+000 AT NODE        0  SPEEDMAX =  0.0000E+000 AT NODE        0
INFO: writeHotstart:  HOT START OUTPUT WRITTEN TO UNIT 67 AT TIME STEP =    172058 TIME =  0.43199996E+06
ok
 INFO: The ROOTDIR is  '.'.
 INFO: The INPUTDIR is '.'.
 INFO: The GBLINPUTDIR is '.'.
 INFO: The GLOBALDIR is '.'.
 INFO: The LOCALDIR is '.'.
ERROR: logNameListReadStatus: Could not read metControl namelist. The Fortran i/o error code was 5010.

 _______________________________________________________________________________


                   PROGRAM ADCIRC   VERSION 454104d

     AN ADVANCED CIRCULATION MODEL FOR SHELVES, COASTAL SEAS AND ESTUARIES


       -  DEVELOPED BY

          R.A. LUETTICH, JR
            UNIVERSITY OF NORTH CAROLINA

Exception: 
At line 2309 of file /Users/simon/adcirc-swan/adcirc/src/wind.F (unit = 22, file = './fort.22')
Fortran runtime error: End of file

Error termination. Backtrace:
#0  0x1012bb187
#1  0x1012bbd37
#2  0x1012bc613
#3  0x1013a5dc3
#4  0x10139fe9f
#5  0x1013a082f
#6  0x1005ca757
#7  0x1006a7dab
#8  0x10066f41b
#9  0x1006e9ddb


In [14]:
?AtmosphericMeshForcing

In [15]:
?AdcircMesh